In [37]:
import pandas as pd
import numpy as np

In [21]:

df = pd.read_csv('./datasets/transactions.csv')

In [22]:
df.shape

(6362620, 11)

In [23]:
df = df[(df["isFraud"] == 1) | (df["isFlaggedFraud"] == 1) | 
            ((df["amount"] != 0) & (df["oldbalanceOrg"] - df["newbalanceOrig"] == df["amount"]) & 
             ((df["newbalanceDest"] - df["oldbalanceDest"] == df["amount"]) | (df["nameDest"].str.contains("M")))) |
            ((df["amount"] != 0) & (df["newbalanceOrig"] - df["oldbalanceOrg"] == df["amount"]) & ((df["oldbalanceDest"] - df["newbalanceDest"] == df["amount"])))]

In [24]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0
12,1,PAYMENT,2560.74,C1648232591,5070.0,2509.26,M972865270,0.0,0.0,0,0
26,1,PAYMENT,9478.39,C1671590089,116494.0,107015.61,M58488213,0.0,0.0,0,0


In [26]:
df.shape

(414089, 11)

In [102]:
# choose a specific date (timestamp) and start doing the step 1 hour count from that timestamp
df['date'] = pd.to_datetime(df['step'], unit='h', origin='2025-01-01')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,date
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,2025-01-01 01:00:00
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,2025-01-01 01:00:00
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,2025-01-01 01:00:00
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,2025-01-01 01:00:00
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,2025-01-01 01:00:00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [103]:
# reset index
df = df.reset_index(drop=True)

In [107]:
import pandas as pd
import numpy as np

class FeatureBuilder:
    def build_customer_dataset(self, df):
        print("Transforming transactions into customer features...")

        # 1. Setup Time Variables
        # PaySim 'step' is hours. 24 steps = 1 day.
        df['day'] = (df['step'] // 24) + 1

        # 2. Aggregate Basic Stats
        # This covers: count, total, average, and max amounts
        customer_df = df.groupby('nameOrig').agg({
            'amount': ['count', 'sum', 'mean', 'max'],
            'step': ['max', 'min']
        })

        # Flatten the MultiIndex columns (e.g., 'amount_sum')
        customer_df.columns = [
            'transaction_count', 
            'total_amount', 
            'average_amount', 
            'max_amount', 
            'last_active_step', 
            'first_active_step'
        ]

        # 3. Calculate Daily Transaction Velocity
        # Velocity = Total Transactions / Total Days present in dataset
        total_days = (df['day'].max() - df['day'].min()) + 1
        customer_df['daily_velocity'] = customer_df['transaction_count'] / total_days

        # 4. Rolling Statistics (Behavioral Shifts)
        # We calculate the rolling average of the last 3 transactions for each user
        # Note: This is done on the original df then mapped back
        df['rolling_avg_3'] = df.groupby('nameOrig')['amount'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
        
        # Take the most recent rolling average per customer as a feature
        latest_rolling = df.groupby('nameOrig')['rolling_avg_3'].last()
        customer_df['current_rolling_avg'] = latest_rolling

        # 5. Save the new dataset
        customer_df.to_csv('customer_features_dataset.csv')
        print("Success! Created 'customer_features_dataset.csv'")
        
        return customer_df

# Usage:
# builder = FeatureBuilder()
# customer_data = builder.build_customer_dataset(df)

FeatureBuilder = FeatureBuilder()
customers_score = FeatureBuilder.build_customer_dataset(df)

Transforming transactions into customer features...
Success! Created 'customer_features_dataset.csv'


In [108]:
customers_score.head()

,transaction_count,total_amount,average_amount,max_amount,last_active_step,first_active_step,daily_velocity,current_rolling_avg
nameOrig,,,,,,,,
C1000000639,1,244486.46,244486.46,244486.46,249,249,0.032258,244486.46
C1000001337,1,3170.28,3170.28,3170.28,217,217,0.032258,3170.28
C1000001725,1,8424.74,8424.74,8424.74,46,46,0.032258,8424.74
C1000002591,1,261877.19,261877.19,261877.19,231,231,0.032258,261877.19
C1000003372,1,20528.65,20528.65,20528.65,167,167,0.032258,20528.65


In [115]:
customers_score.daily_velocity.value_counts()

daily_velocity
0.032258    6344009
0.064516       9283
0.096774         15
Name: count, dtype: int64

In [109]:
customers_score.transaction_count.value_counts()

transaction_count
1    6344009
2       9283
3         15
Name: count, dtype: int64

In [110]:
customers_score.daily_velocity.value_counts()

daily_velocity
0.032258    6344009
0.064516       9283
0.096774         15
Name: count, dtype: int64

In [111]:
customers_score.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6353307 entries, C1000000639 to C999999784
Data columns (total 8 columns):
 #   Column               Dtype  
---  ------               -----  
 0   transaction_count    int64  
 1   total_amount         float64
 2   average_amount       float64
 3   max_amount           float64
 4   last_active_step     int64  
 5   first_active_step    int64  
 6   daily_velocity       float64
 7   current_rolling_avg  float64
dtypes: float64(5), int64(3)
memory usage: 436.2+ MB


In [113]:
# print the value counts of the index of the data

customers_score.index.value_counts()

nameOrig
C1000000639    1
C1000001337    1
C1000001725    1
C1000002591    1
C1000003372    1
              ..
C999996999     1
C999998175     1
C999999254     1
C999999614     1
C999999784     1
Name: count, Length: 6353307, dtype: int64

In [114]:
customers_score.describe()

,transaction_count,total_amount,average_amount,max_amount,last_active_step,first_active_step,daily_velocity,current_rolling_avg
count,6.353307e+06,6.353307e+06,6.353307e+06,6.353307e+06,6.353307e+06,6.353307e+06,6.353307e+06,6.353307e+06
mean,1.001466e+00,1.801256e+05,1.798571e+05,1.800448e+05,2.435164e+02,2.432844e+02,3.230535e-02,1.798571e+05
std,3.832002e-02,6.043381e+05,6.034020e+05,6.042734e+05,1.423390e+02,1.423266e+02,1.236130e-03,6.034020e+05
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,3.225806e-02,0.000000e+00
25%,1.000000e+00,1.341488e+04,1.340753e+04,1.341194e+04,1.560000e+02,1.550000e+02,3.225806e-02,1.340753e+04
50%,1.000000e+00,7.506847e+04,7.497439e+04,7.504769e+04,2.390000e+02,2.390000e+02,3.225806e-02,7.497439e+04
75%,1.000000e+00,2.089948e+05,2.086982e+05,2.089080e+05,3.350000e+02,3.340000e+02,3.225806e-02,2.086982e+05
max,3.000000e+00,9.244552e+07,9.244552e+07,9.244552e+07,7.430000e+02,7.430000e+02,9.677419e-02,9.244552e+07


In [ ]:
def apply_risk_scoring(customer_df):
        print("Applying log transformation and calculating risk scores...")
        
        # 1. Target features for scoring
        features_to_score = ['total_amount', 'daily_velocity', 'current_rolling_avg']
        
        for feature in features_to_score:
            # 2. Overwrite the original column with its log value
            # We use log1p (log of 1+x) to avoid errors with 0 values
            customer_df[feature] = np.log1p(customer_df[feature])
            
            # 3. Calculate Z-score based on the now-transformed column
            mu = customer_df[feature].mean()
            sigma = customer_df[feature].std()
            
            # Note: We still create a separate Z-score column to keep the logic clean
            customer_df[f'{feature}_zscore'] = (customer_df[feature] - mu) / sigma
        
        # 4. Determine the Maximum Z-score across these factors
        z_cols = [f'{f}_zscore' for f in features_to_score]
        customer_df['max_z'] = customer_df[z_cols].max(axis=1)
        
        # 5. Classify Risk Bands
        def classify(z):
            if z > 3: return 'Critical'
            if z > 1.5: return 'High'
            if z > 0: return 'Medium'
            return 'Low'
        
        customer_df['risk_band'] = customer_df['max_z'].apply(classify)
        
        print("Risk Analysis Complete.")
        return customer_df


customers_score = apply_risk_scoring(customers_score)
customers_score.head()



Applying log transformation and calculating risk scores...
Risk Analysis Complete.


,transaction_count,total_amount,average_amount,max_amount,last_active_step,first_active_step,daily_velocity,current_rolling_avg,total_amount_zscore,daily_velocity_zscore,current_rolling_avg_zscore,max_z,risk_band
nameOrig,,,,,,,,,,,,,
C1000000639,1,12.406919,244486.46,244486.46,249,249,0.031749,12.406919,0.862087,-0.038255,0.862884,0.862884,Medium
C1000001337,1,8.061891,3170.28,3170.28,217,217,0.031749,8.061891,-1.532616,-0.038255,-1.532478,-0.038255,Low
C1000001725,1,9.039047,8424.74,8424.74,46,46,0.031749,9.039047,-0.994070,-0.038255,-0.993784,-0.038255,Low
C1000002591,1,12.475635,261877.19,261877.19,231,231,0.031749,12.475635,0.899959,-0.038255,0.900766,0.900766,Medium
C1000003372,1,9.929625,20528.65,20528.65,167,167,0.031749,9.929625,-0.503239,-0.038255,-0.502818,-0.038255,Low


In [117]:
# save user nameorig and his risk band to a csv file

customers_score[['risk_band']].to_csv('customer_risk_bands.csv')


In [ ]:
customers_score.head()

,transaction_count,total_amount,average_amount,max_amount,last_active_step,first_active_step,daily_velocity,current_rolling_avg,total_amount_zscore,daily_velocity_zscore,current_rolling_avg_zscore,max_z,risk_band
nameOrig,,,,,,,,,,,,,
C1000000639,1,12.406919,244486.46,244486.46,249,249,0.031749,12.406919,0.862087,-0.038255,0.862884,0.862884,Medium
C1000001337,1,8.061891,3170.28,3170.28,217,217,0.031749,8.061891,-1.532616,-0.038255,-1.532478,-0.038255,Low
C1000001725,1,9.039047,8424.74,8424.74,46,46,0.031749,9.039047,-0.994070,-0.038255,-0.993784,-0.038255,Low
C1000002591,1,12.475635,261877.19,261877.19,231,231,0.031749,12.475635,0.899959,-0.038255,0.900766,0.900766,Medium
C1000003372,1,9.929625,20528.65,20528.65,167,167,0.031749,9.929625,-0.503239,-0.038255,-0.502818,-0.038255,Low


In [2]:
import pandas as pd
df = pd.read_csv('customer_risk_bands.csv')

In [3]:
df.head()

,nameOrig,risk_band
0,C1000000639,Medium
1,C1000001337,Low
2,C1000001725,Low
3,C1000002591,Medium
4,C1000003372,Low
